In [ ]:
# -*- coding: utf-8 -*-
"""SegNet model for Keras.
# Reference:
- [Segnet: A deep convolutional encoder-decoder architecture for image segmentation](https://arxiv.org/pdf/1511.00561.pdf)
"""

from __future__ import absolute_import
from __future__ import print_function
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
import numpy as np
from keras.applications import imagenet_utils
from keras.utils import np_utils
from keras.layers import Input
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def load_data(labeled_path, unlabeled_path):

	# Find labeled files
	labeled_files = [f for f in listdir(labeled_path) if isfile(join(labeled_path,f))]
	# Find unlabeled files
	unlabeled_files = [f for f in listdir(unlabeled_path) if isfile(join(unlabeled_path,f))]

	# Allocate array for labeled images
	images_labeled = np.empty([len(labeled_files), 360, 480], dtype = 'uint8')

	# Allocate array for unlabeled images
	images_unlabeled = np.empty([len(unlabeled_files), 360, 480, 3], dtype = 'uint8')


	# Reading labeled images
	for n in range(0, len(labeled_files)):
	    images_labeled[n] = cv2.imread(join(labeled_path,labeled_files[n]), 0)

	# Reading unlabeled images
	for n in range(0, len(unlabeled_files)):
	    images_unlabeled[n] = cv2.imread(join(unlabeled_path,unlabeled_files[n]), 1)

	# Preprocess unlabeled images
	images_unlabeled = images_unlabeled.astype('float64')
	X = imagenet_utils.preprocess_input(images_unlabeled)

	# Preprocess labeled images
	n_labeled = len(labeled_files)
	Y = np_utils.to_categorical(images_labeled.flatten(), 12)
	Y = Y.reshape((n_labeled, images_labeled.size / n_labeled, 12))

	return X, Y

def SegNet(input_shape=(360, 480, 3), classes=12):
    # c.f. https://github.com/alexgkendall/SegNet-Tutorial/blob/master/Example_Models/bayesian_segnet_camvid.prototxt
    img_input = Input(shape=input_shape)
    x = img_input
    # Encoder
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(256, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    # Decoder
    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(classes, (1, 1), padding="valid")(x)
    x = Reshape((input_shape[0]*input_shape[1], classes))(x)
    x = Activation("softmax")(x)
    model = Model(img_input, x)
    return model


In [ ]:
input_shape = (360, 480, 3)
batch_size = 1
epochs = 100

train_labeled_path = '/home/zaziza/virtualenv/project_space/SegNet/CamVid/trainannot'
train_unlabeled_path = '/home/zaziza/virtualenv/project_space/SegNet/CamVid/train'

test_labeled_path = '/home/zaziza/virtualenv/project_space/SegNet/CamVid/testannot'
test_unlabeled_path = '/home/zaziza/virtualenv/project_space/SegNet/CamVid/test'


print "Loading training dataset"
X, Y = load_data(train_labeled_path, train_unlabeled_path)
print "Dataset has been loaded"
model = SegNet(input_shape=input_shape, classes=12)
model.compile(loss="categorical_crossentropy", optimizer='adadelta', metrics=["accuracy"])

#start = timeit.default_timer()
print "Started training"
model.fit(X, Y, batch_size=batch_size, epochs = epochs)
print "Training has finished"
#stop = timeit.default_time()

#print stop - start
model.save('trained_model')

print "Loading testing dataset"
X_test, Y_test = load_data(test_labeled_path, test_unlabeled_path)
print "Dataset has been loaded"
print "Evaluating"
model.evaluate(X_test, Y_test, verbose=0)